In [1]:
import pandas as pd

num = pd.read_csv('bosch_small_data/train_numeric.csv')

#go to neo4j.conf file
# uncomment dbms.security.allow_csv_import_from_file_urls=true

#change import folder
#dbms.directories.import=/Users/julieshih/workspace/supply_chain/neo4j 
#put all csv files into neo4j folder

#NEOJ4 database location
#/Users/julieshih/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-dd992aba-8cf8-4438-9696-fcf5516178bd/installation-3.5.0/import

In [26]:
#get station names only
df = num.iloc[:,1:-1] 
stations = list(df.columns.values)
stations[:10] #save this list to nodes.csv

['L0_S0_F0',
 'L0_S0_F2',
 'L0_S0_F4',
 'L0_S0_F6',
 'L0_S0_F8',
 'L0_S0_F10',
 'L0_S0_F12',
 'L0_S0_F14',
 'L0_S0_F16',
 'L0_S0_F18']

In [28]:
nodelist = pd.read_csv('nodelist.csv')
split = pd.DataFrame(nodes.stationId.str.split('_',2).tolist(),
                                   columns = ['line','station','feature'])
split.head()

,line,station,feature
0,L0,S0,F0
1,L0,S0,F2
2,L0,S0,F4
3,L0,S0,F6
4,L0,S0,F8


In [22]:
nodes = pd.concat([nodelist,split],axis=1)
nodes.to_csv('nodes.csv',index=False)
nodes.head()

,stationId,line,station,feature
0,L0_S0_F0,L0,S0,F0
1,L0_S0_F2,L0,S0,F2
2,L0_S0_F4,L0,S0,F4
3,L0_S0_F6,L0,S0,F6
4,L0_S0_F8,L0,S0,F8


In [7]:
#create a node in graph database to represent each station
#for station in stations:
#    query = 'CREATE (s:Station {stationId:"' + station +'"})'
#    print(query)

In [ ]:
#create station nodes

# assert correct line count
LOAD CSV FROM "file:///nodes.csv" AS line
RETURN count(*);

# check first few raw lines
LOAD CSV FROM "file:///nodes.csv" AS line WITH line
RETURN line
LIMIT 5;

#check first 5 lines with header mapping
LOAD CSV WITH HEADERS FROM "file:///nodes.csv" AS line WITH line
RETURN line
LIMIT 5;

#create the nodes
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///nodes.csv" AS row
CREATE (:Station {stationId: row.stationId, line:row.line, station: row.station, feature: row.feature});

#ensure station nodes are unique
CREATE CONSTRAINT ON (s:Station) ASSERT s.stationId IS UNIQUE;

#create index to make matching faster
CREATE INDEX ON :Station(feature);

In [30]:
#load data of parsed transitions
parsed = pd.read_csv('relationships.csv')
parsed.iloc[200:250,:]

,enter,exit,line_start,line_end,value,part_id,response
200,L0_S11_F322,L0_S11_F326,L0_S0_F0,L3_S37_F3950,0.053,71,0.0
201,L0_S11_F326,L3_S29_F3315,L0_S0_F0,L3_S37_F3950,-0.039,71,0.0
202,L3_S29_F3315,L3_S29_F3318,L0_S0_F0,L3_S37_F3950,0.448,71,0.0
203,L3_S29_F3318,L3_S29_F3321,L0_S0_F0,L3_S37_F3950,0.077,71,0.0
204,L3_S29_F3321,L3_S29_F3324,L0_S0_F0,L3_S37_F3950,0.011,71,0.0
205,L3_S29_F3324,L3_S29_F3327,L0_S0_F0,L3_S37_F3950,-0.146,71,0.0
206,L3_S29_F3327,L3_S29_F3330,L0_S0_F0,L3_S37_F3950,-0.017,71,0.0
207,L3_S29_F3330,L3_S29_F3333,L0_S0_F0,L3_S37_F3950,-0.018,71,0.0
208,L3_S29_F3333,L3_S29_F3336,L0_S0_F0,L3_S37_F3950,0.013,71,0.0
209,L3_S29_F3336,L3_S29_F3339,L0_S0_F0,L3_S37_F3950,0.038,71,0.0


In [ ]:
#parse station name down to feature level


In [ ]:
#test first
#LOAD CSV FROM "file:///nodes.csv" AS line WITH line
#RETURN line
#LIMIT 5;


#check first few rows
LOAD CSV WITH HEADERS FROM "file:///relationships.csv" AS row
RETURN row
LIMIT 5;

#try adding relationships to first few rows
LOAD CSV WITH HEADERS FROM "file:///relationships" AS row
MATCH (en:Station {stationId: row.enter})
MATCH (ex:Station {stationId: row.exit})
MERGE (en)-[r:TO {lineStart:row.line_start,lineEnd:row.line_end,value:row.value,partID: row.part_id,response:row.response}]->(ex)
RETURN en, ex
LIMIT 50;


#delete previously created relationships:
#match (n)-[r]-(m) delete r
#create all relationships
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///relationships" AS row
MATCH (en:Station {stationId: row.enter})
MATCH (ex:Station {stationId: row.exit})
MERGE (en)-[r:TO {lineStart:row.line_start,lineEnd:row.line_end,value:row.value,partID: row.part_id,response:row.response}]->(ex)